## Bayesian Modeling ##

In [1]:
import os
import sys
import importlib
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm

In [2]:
# 设定项目根目录
project_root = Path(os.getcwd()).parent
sys.path.append(str(project_root))

In [3]:
# 导入数据
processed_path = Path(project_root) / 'data' / 'processed'
learning_data = pd.read_csv(processed_path / 'Task2_processed.csv')

In [4]:
# 导入模型
from src.Bayesian_new import *

### 0. Unit Test

#### 0.1 Base Test

In [5]:
p = Partition(4,4)

In [6]:
p.prototypes

[('2d_axis_pair',
  {0: (0.25, 0.25, 0.5, 0.5),
   1: (0.75, 0.25, 0.5, 0.5),
   2: (0.25, 0.75, 0.5, 0.5),
   3: (0.75, 0.75, 0.5, 0.5)}),
 ('2d_axis_pair',
  {0: (0.25, 0.5, 0.25, 0.5),
   1: (0.75, 0.5, 0.25, 0.5),
   2: (0.25, 0.5, 0.75, 0.5),
   3: (0.75, 0.5, 0.75, 0.5)}),
 ('2d_axis_pair',
  {0: (0.25, 0.5, 0.5, 0.25),
   1: (0.75, 0.5, 0.5, 0.25),
   2: (0.25, 0.5, 0.5, 0.75),
   3: (0.75, 0.5, 0.5, 0.75)}),
 ('2d_axis_pair',
  {0: (0.5, 0.25, 0.25, 0.5),
   1: (0.5, 0.75, 0.25, 0.5),
   2: (0.5, 0.25, 0.75, 0.5),
   3: (0.5, 0.75, 0.75, 0.5)}),
 ('2d_axis_pair',
  {0: (0.5, 0.25, 0.5, 0.25),
   1: (0.5, 0.75, 0.5, 0.25),
   2: (0.5, 0.25, 0.5, 0.75),
   3: (0.5, 0.75, 0.5, 0.75)}),
 ('2d_axis_pair',
  {0: (0.5, 0.5, 0.25, 0.25),
   1: (0.5, 0.5, 0.75, 0.25),
   2: (0.5, 0.5, 0.25, 0.75),
   3: (0.5, 0.5, 0.75, 0.75)}),
 ('2d_equality_sum',
  {0: (0.5, 0.16666666666666666, 0.5, 0.5),
   1: (0.5, 0.8333333333333334, 0.5, 0.5),
   2: (0.8333333333333334, 0.5, 0.5, 0.5),
   3: (0.

In [7]:
p.calc_likelihood_entry(5, (np.array([[0.1,0.2,0.3,0.4],[0.1,0.4,0.3,0.2],[0.4,0.2,0.3,0.1]]), np.array([3,2,3]), np.array([1,0,1])), 2.)

array([0.26337826, 0.74936102, 0.19091061])

In [10]:
p.calc_likelihood_entry(2, (np.array([[0.1,0.2,0.3,0.4],[0.1,0.2,0.3,0.4],[0.1,0.2,0.3,0.4]]), np.array([3,2,3]), np.array([1,0,1])), 2.)

array([0.29285594, 0.81655506, 0.29285594])

In [11]:
pl = PartitionLikelihood(BaseSet(range(10)), p)

In [12]:
pl.h_set

Base Set of (index, value)'s:
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [10]:
pl.get_likelihood((np.array([[0.1,0.2,0.3,0.4],[0.05,0.02,0.53,0.4],[0.1,0.2,0.3,0.4]]), np.array([3,2,3]), np.array([1,1,0])), beta=15., normalized=True)

array([[0.00538931, 0.00406884, 0.00266818, 0.01182219, 0.00833538,
        0.02414705, 0.20918334, 0.24987487, 0.27611911, 0.20839173],
       [0.00624016, 0.21553695, 0.11519967, 0.21592538, 0.1161825 ,
        0.04417619, 0.2087503 , 0.01871472, 0.04304554, 0.0162286 ],
       [0.1425153 , 0.14310868, 0.1437381 , 0.13962455, 0.14119142,
        0.13408612, 0.05093617, 0.03265058, 0.02085718, 0.0512919 ]])

In [11]:
spl = SoftPartitionLikelihood(BaseSet(range(10)), p, np.exp(np.linspace(0,5,10)))

In [ ]:
spl.get_likelihood((np.array([[0.1,0.2,0.3,0.4],[0.05,0.02,0.53,0.4],[0.1,0.2,0.3,0.4]]), np.array([3,2,3]), np.array([1,1,0])), normalized=True)

In [13]:
learning_data

,iSub,version,condition,iSession,iBlock,iTrial,neck_length,head_length,leg_length,tail_length,...,leg_angle,tail_angle,feature1,feature2,feature3,feature4,category,choice,feedback,choRT
0,1,1,1,1,1,1,0.474262,0.484076,0.096722,0.080176,...,0.5,0.5,0.096722,0.484076,0.080176,0.474262,1,1,1.0,15.639
1,1,1,1,1,1,2,0.556726,0.976765,0.280363,0.079595,...,0.5,0.5,0.280363,0.976765,0.079595,0.556726,2,1,1.0,31.626
2,1,1,1,1,1,3,0.154921,0.514583,0.335823,0.299767,...,0.5,0.5,0.335823,0.514583,0.299767,0.154921,2,1,1.0,37.057
3,1,1,1,1,1,4,0.554234,0.225965,0.892292,0.500937,...,0.5,0.5,0.892292,0.225965,0.500937,0.554234,4,2,1.0,6.335
4,1,1,1,1,1,5,0.215180,0.850173,0.891455,0.639999,...,0.5,0.5,0.891455,0.850173,0.639999,0.215180,4,2,1.0,7.111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11195,24,1,3,1,2,124,0.441382,0.785711,0.023709,0.153259,...,0.5,0.5,0.441382,0.785711,0.023709,0.153259,2,2,1.0,12.196
11196,24,1,3,1,2,125,0.866531,0.222335,0.289970,0.747357,...,0.5,0.5,0.866531,0.222335,0.289970,0.747357,3,3,1.0,5.314
11197,24,1,3,1,2,126,0.946735,0.187566,0.890659,0.816365,...,0.5,0.5,0.946735,0.187566,0.890659,0.816365,4,4,1.0,5.929
11198,24,1,3,1,2,127,0.645122,0.751123,0.497858,0.295268,...,0.5,0.5,0.645122,0.751123,0.497858,0.295268,3,4,0.5,17.862


#### 0.2 Model Test

#### (1) Base model

In [46]:
import src.Bayesian_new.problems.model as model
importlib.reload(model)
from src.Bayesian_new.problems.model import SingleRationalModel

import src.Bayesian_new.problems.config as config
importlib.reload(config)
from src.Bayesian_new.problems.config import config_base

base_model = SingleRationalModel(config_base)

In [47]:
# 模型拟合
fitting_results = {}

for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
    if iSub>1:
        break
    condition = subject_data['condition'].iloc[0]
    model = SingleRationalModel(config_base, condition=condition)
    print(iSub)
    s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
              subject_data["choice"].values,
              subject_data["feedback"].values)
    # print(subject_data)
    step_results = base_model.fit_trial_by_trial(s_data)
    fitting_results[iSub] = {
        'condition': condition,
        'step_results': step_results
    }

1


100%|██████████| 128/128 [00:01<00:00, 70.59it/s]


In [48]:
fitting_results

{1: {'condition': 1,
  'step_results': [{'best_k': 11,
    'best_beta': 23.838387787129815,
    'best_params': ModelParams(k=11, beta=23.838387787129815),
    'best_log_likelihood': -1.3447557707705544e-05,
    'best_norm_posterior': 0.060214347117609264,
    'hypo_details': {0: {'beta_opt': 30.0,
      'll_max': -5.1418805503297094e-05,
      'post_max': 0.05689316992061488,
      'is_best': False},
     1: {'beta_opt': 30.0,
      'll_max': -0.5224648138088099,
      'post_max': 0.04922125984947686,
      'is_best': False},
     2: {'beta_opt': 30.0,
      'll_max': -3.314678262041502e-05,
      'post_max': 0.05724116808176898,
      'is_best': False},
     3: {'beta_opt': 30.0,
      'll_max': -0.4343919915554573,
      'post_max': 0.049410499815036264,
      'is_best': False},
     4: {'beta_opt': 30.0,
      'll_max': -0.0019349888987891038,
      'post_max': 0.053988442051264275,
      'is_best': False},
     5: {'beta_opt': 0.001,
      'll_max': -0.6931515663171384,
      'post

In [ ]:
sub_results = fitting_results[1]

In [ ]:
step_results = sub_results['step_results']

#### (2) Forget model

In [5]:
import src.Bayesian_new.problems.forget as forget
importlib.reload(forget)
from src.Bayesian_new.problems.forget import ForgetModel

import src.Bayesian_new.problems.config as config
importlib.reload(config)
from src.Bayesian_new.problems.config import config_fgt

forget_model = ForgetModel(config_fgt)

In [6]:
# 模型拟合
fitting_results = {}

for i, (iSub, subject_data) in enumerate(learning_data.groupby('iSub')):
    if iSub>1:
        break
    condition = subject_data['condition'].iloc[0]
    model = ForgetModel(config_fgt, condition=condition)
    print(iSub)
    s_data = (subject_data[["feature1", "feature2", "feature3", "feature4"]].values,
              subject_data["choice"].values,
              subject_data["feedback"].values, 
              subject_data["category"].values)
    # print(subject_data)
    best_params, step_results = forget_model.fit(s_data)
    # step_results = model.fit_trial_by_trial(s_data)
    fitting_results[iSub] = {
        'condition': condition,
        'best_params': best_params,
        'step_results': step_results
    }

1


0it [00:00, ?it/s]    | 0/10 [00:00<?, ?it/s]
Gamma:   0%|          | 0/10 [00:00<?, ?it/s]


KeyError: 0

### 1. Base/ rational ###

In [7]:
# 导入基线模型
import src.Bayesian.M_base_new as model_base
importlib.reload(model_base)
from src.Bayesian.M_base_new import M_Base

import src.Bayesian.config as config
importlib.reload(config)
from src.Bayesian.config import config_base
model_base = M_Base(config_base)

model_base.set_centers(all_centers)

In [9]:
# 模型拟合
fitting_results = {}
for iSub, subject_data in learning_data.groupby('iSub'):
    if iSub>2:
        break
    step_results = model_base.fit_trial_by_trial(subject_data)
    fitting_results[iSub] = {
        'condition': subject_data['condition'].iloc[0],
        'step_results': step_results
    }

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703/703 [03:17<00:00,  3.56it/s]


In [11]:
fitting_results

{1: {'condition': 1,
  'step_results': [{'k': 12,
    'beta': 23.828959936585232,
    'best_log_likelihood': -1.3507344504298421e-05,
    'best_posterior': -2.944452486510945,
    'k_posteriors': {1: 0.06497064837129678,
     2: 0.03853327714476673,
     3: 0.06497183552731636,
     4: 0.04208094526258652,
     5: 0.06484838678136737,
     6: 0.03248685210540126,
     7: 0.06482625018564282,
     8: 0.03248346336395167,
     9: 0.0324869100768721,
     10: 0.06486164941349595,
     11: 0.06490004064387801,
     12: 0.06497311155018964,
     13: 0.06491123652540358,
     14: 0.06491793464228197,
     15: 0.04287418590562322,
     16: 0.06492645817543895,
     17: 0.03248678203368774,
     18: 0.06497309198611617,
     19: 0.03248694030468303},
    'params': ModelParams(k=12, beta=23.828959936585232)},
   {'k': 18,
    'beta': 27.020530892735653,
    'best_log_likelihood': -1.699052402807012e-05,
    'best_posterior': -2.944455969690469,
    'k_posteriors': {1: 0.0843750520898843,
     2

In [ ]:
# 保存拟合结果
result_path = Path(project_root) / 'results' / 'Bayesian'
os.makedirs(result_path, exist_ok=True)

filename = f'M_Base_fitting_results.joblib'
joblib.dump(fitting_results, result_path / filename)

In [ ]:
# 导入模型分析
import src.Bayesian.utils.model_evaluation as model_eval
importlib.reload(model_eval)
from src.Bayesian.utils.model_evaluation import ModelEval

model_eval = ModelEval()

In [ ]:
# 加载拟合结果
fitting_results = joblib.load(result_path / 'M_Base_fitting_results.joblib')

# 绘制最优参数变化图
model_eval.plot_params_over_trials(fitting_results, 'beta', result_path / f'M_Base_beta.png')

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(fitting_results, result_path / f'M_Base_posteriors.png')

# 计算预测准确率
predictive_results = {}
for iSub, fit_result in fitting_results.items():
    subject_data = learning_data[learning_data['iSub'] == iSub]
    step_results = fit_result['step_results']
    condition = fit_result['condition']
    
    predictions = model_eval.calculate_predictions(model_base, subject_data, step_results)
    sliding_accuracy = model_eval.calculate_sliding_accuracy(predictions)
    
    predictive_results[iSub] = {
        'condition': condition,
        'step_results': step_results,
        'predictions': predictions,
        'sliding_accuracy': sliding_accuracy
    }

# 绘制预测准确率变化图
filename = f'M_Base_predictive_accuracy.png'
model_eval.plot_predictive_accuracy(predictive_results, result_path / filename)

### 2. + Forgetting ###

In [ ]:
# 导入模型
import src.Bayesian.M_fgt as model_forget
importlib.reload(model_forget)
from src.Bayesian.M_fgt import M_Fgt

import src.Bayesian.config as config
importlib.reload(config)
from src.Bayesian.config import config_fgt
model_forget = M_Fgt(config_fgt)

model_forget.set_centers(all_centers)

In [ ]:
fitting_results = {}
for iSub, subject_data in learning_data.groupby('iSub'):
    step_results = model_forget.fit_trial_by_trial(subject_data)
    fitting_results[iSub] = {
        'condition': subject_data['condition'].iloc[0],
        'step_results': step_results
    }

In [ ]:
# 保存拟合结果
result_path = Path(project_root) / 'results' / 'Bayesian'
os.makedirs(result_path, exist_ok=True)

filename = f'M_Fgt_fitting_results.joblib'
joblib.dump(fitting_results, result_path / filename)

In [ ]:
# 加载拟合结果
fitting_results = joblib.load(result_path / 'M_Fgt_fitting_results.joblib')

# 绘制后验概率变化图
filename = f'M_Fgt_posteriors.png'
model_eval.plot_posterior_probabilities(fitting_results, result_path / filename)

# 计算预测准确率
predictive_results = {}
for iSub, fit_result in fitting_results.items():
    try:
        subject_data = learning_data[learning_data['iSub'] == iSub]
        step_results = fit_result['step_results']
        condition = fit_result['condition']
        
        predictions = model_eval.calculate_predictions(model_forget, subject_data, step_results)
        sliding_accuracy = model_eval.calculate_sliding_accuracy(predictions)
        
        predictive_results[iSub] = {
            'condition': condition,
            'step_results': step_results,
            'predictions': predictions,
            'sliding_accuracy': sliding_accuracy
        }
    except Exception as e:
        print(f"Error processing predictions for subject {iSub}: {str(e)}")
        continue

# 绘制预测准确率变化图
filename = f'M_Fgt_predictive_accuracy.png'
model_eval.plot_predictive_accuracy(predictive_results, result_path / filename)

### 3. + Decision Noise ###

In [ ]:
# 导入模型
import src.Bayesian.M_dec as model_decision
importlib.reload(model_decision)
from src.Bayesian.M_dec import M_Dec

import src.Bayesian.config as config
importlib.reload(config)
from src.Bayesian.config import config_dec
model_decision = M_Dec(config_dec)

model_decision.set_centers(all_centers)

In [ ]:
# 模型拟合
fitting_results = {}
for iSub, subject_data in learning_data.groupby('iSub'):
    step_results = model_decision.fit_trial_by_trial(subject_data)
    fitting_results[iSub] = {
        'condition': subject_data['condition'].iloc[0],
        'step_results': step_results
    }

In [ ]:
# 保存拟合结果
result_path = Path(project_root) / 'results' / 'Bayesian'
os.makedirs(result_path, exist_ok=True)

filename = f'M_Dec_fitting_results.joblib'
joblib.dump(fitting_results, result_path / filename)

In [ ]:
# 加载拟合结果
fitting_results = joblib.load(result_path / 'M_Dec_fitting_results.joblib')

# 绘制最优参数变化图
model_eval.plot_params_over_trials(fitting_results, 'beta', result_path / f'M_Dec_beta.png')
model_eval.plot_params_over_trials(fitting_results, 'phi', result_path / f'M_Dec_phi.png')

# 绘制k的后验概率变化图
model_eval.plot_posterior_probabilities(fitting_results, result_path / f'M_Dec_posteriors.png')

# 计算预测准确率
predictive_results = {}
for iSub, fit_result in fitting_results.items():
    try:
        subject_data = learning_data[learning_data['iSub'] == iSub]
        step_results = fit_result['step_results']
        condition = fit_result['condition']
        
        predictions = model_eval.calculate_predictions(model_decision, subject_data, step_results)
        sliding_accuracy = model_eval.calculate_sliding_accuracy(predictions)
        
        predictive_results[iSub] = {
            'condition': condition,
            'step_results': step_results,
            'predictions': predictions,
            'sliding_accuracy': sliding_accuracy
        }
    except Exception as e:
        print(f"Error processing predictions for subject {iSub}: {str(e)}")
        continue

# 绘制预测准确率变化图
filename = f'M_Dec_predictive_accuracy.png'
model_eval.plot_predictive_accuracy(predictive_results, result_path / filename)

### 4. + Perceptive Noise ###

In [ ]:
learning_data = pd.read_csv(processed_path / 'Task2_processed_perceived.csv')

In [ ]:
# 导入基线模型
import src.Bayesian.M_base as model_base
importlib.reload(model_base)
from src.Bayesian.M_base import M_Base

import src.Bayesian.config as config
importlib.reload(config)
from src.Bayesian.config import config_base
model_base = M_Base(config_base)

model_base.set_centers(all_centers)

In [ ]:
# 模型拟合
fitting_results = {}
for iSub, subject_data in learning_data.groupby('iSub'):
    step_results = model_base.fit_trial_by_trial(subject_data)
    fitting_results[iSub] = {
        'condition': subject_data['condition'].iloc[0],
        'step_results': step_results
    }

In [ ]:
# 保存拟合结果
result_path = Path(project_root) / 'results' / 'Bayesian'
os.makedirs(result_path, exist_ok=True)

filename = f'M_Perc_fitting_results.joblib'
joblib.dump(fitting_results, result_path / filename)

In [ ]:
# 加载拟合结果
fitting_results = joblib.load(result_path / 'M_Perc_fitting_results.joblib')

# 绘制最优参数变化图
model_eval.plot_params_over_trials(fitting_results, 'beta', result_path / f'M_Perc_beta.png')

# 绘制k后验概率变化图
model_eval.plot_posterior_probabilities(fitting_results, result_path / f'M_Perc_posteriors.png')

# 计算预测准确率
predictive_results = {}
for iSub, fit_result in fitting_results.items():
    subject_data = learning_data[learning_data['iSub'] == iSub]
    step_results = fit_result['step_results']
    condition = fit_result['condition']
    
    predictions = model_eval.calculate_predictions(model_base, subject_data, step_results)
    sliding_accuracy = model_eval.calculate_sliding_accuracy(predictions)
    
    predictive_results[iSub] = {
        'condition': condition,
        'step_results': step_results,
        'predictions': predictions,
        'sliding_accuracy': sliding_accuracy
    }

# 绘制预测准确率变化图
filename = f'M_Perc_predictive_accuracy.png'
model_eval.plot_predictive_accuracy(predictive_results, result_path / filename)

### 5. + Attention Weight ###

In [ ]:
# 导入基线模型
import src.Bayesian.M_base as model_base
importlib.reload(model_base)
from src.Bayesian.M_base import M_Base

import src.Bayesian.config as config
importlib.reload(config)
from src.Bayesian.config import config_base
model_base = M_Base(config_base)

model_base.set_centers(all_centers)

### 6. Full ###